### Symbols in Market Watch

In [20]:
import MetaTrader5 as mt5
from dotenv import load_dotenv
import os
import csv

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("AQUA_MT5_LOGIN"))
password = os.getenv("AQUA_MT5_PASSWORD")
server = os.getenv("AQUA_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# Fetch all symbols currently visible in Market Watch
symbols = mt5.symbols_get()

# Filter only active tradable symbols
active_symbols = [
    sym.name
    for sym in symbols
    if sym.visible and sym.trade_mode == mt5.SYMBOL_TRADE_MODE_FULL
]

# Print active symbols
print("Active symbols in Market Watch:")
for name in active_symbols:
    print(name)

# -----------------------------
# Export to CSV
# -----------------------------
csv_filename = "active_symbols.csv"

with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Symbol"])  # header
    for name in active_symbols:
        writer.writerow([name])

print(f"\nExported {len(active_symbols)} symbols to {csv_filename}")

# Shutdown MT5 connection
mt5.shutdown()

Active symbols in Market Watch:
EURUSD
GBPUSD
USDCHF
USDJPY
USDCAD
AUDUSD
NZDUSD
XAGUSD
XAUUSD
BRENT
WTI
AUS200
GER40
JPN225
UK100
US30
SPX500
NAS100

Exported 18 symbols to active_symbols.csv


True

### Max Sharpe Weights

In [21]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from dotenv import load_dotenv
import os

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("AQUA_MT5_LOGIN"))
password = os.getenv("AQUA_MT5_PASSWORD")
server = os.getenv("AQUA_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# -----------------------------
# Config
# -----------------------------
risk_free_rate = 0.0  # annual risk-free rate

# FX pairs needed to convert index returns to USD
fx_map = {
    "GER40": "EURUSD",
    "JPN225": "USDJPY",   # flip sign
    "UK100": "GBPUSD",
    "AUS200": "AUDUSD"
}

# -----------------------------
# Helpers
# -----------------------------
def fetch_mt5_data(symbol, days=60):
    utc_to = pd.Timestamp.now()
    utc_from = utc_to - pd.Timedelta(days=days)

    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, utc_from, utc_to)
    if rates is None:
        raise ValueError(f"Failed to fetch data for {symbol}")

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)
    return df

def calculate_log_returns(prices: pd.Series) -> pd.Series:
    return np.log(prices / prices.shift(1)).dropna()

# -----------------------------
# Load expected returns
# -----------------------------
# ai_factor_signal_aqua.csv:  asset \t factor_signal
expected_returns_df = pd.read_csv("ai_factor_signal_aqua.csv", sep="\t", header=None)
expected_returns_df.columns = ["asset", "factor_signal"]

# -----------------------------
# Z-score the factor signals
# -----------------------------
mean_signal = expected_returns_df["factor_signal"].mean()
std_signal = expected_returns_df["factor_signal"].std()

expected_returns_df["zscore_signal"] = (expected_returns_df["factor_signal"] - mean_signal) / std_signal

# Convert to dict
expected_returns_dict = dict(zip(expected_returns_df["asset"], expected_returns_df["zscore_signal"]))

symbols = list(expected_returns_dict.keys())

# -----------------------------
# Fetch price data for assets
# -----------------------------
data = {symbol: fetch_mt5_data(symbol) for symbol in symbols}

# -----------------------------
# Fetch FX data and compute FX log returns
# -----------------------------
fx_symbols = set(fx_map[s] for s in symbols if s in fx_map)
fx_data = {fx: fetch_mt5_data(fx) for fx in fx_symbols}

fx_returns = {}
for fx, df_fx in fx_data.items():
    lr = calculate_log_returns(df_fx["close"])

    # If pair is quoted as USDXXX (e.g., USDJPY, USDHKD), flip sign to get XXXUSD log return
    if fx.startswith("USD"):
        lr = -lr  # log(1/x) = -log(x)

    fx_returns[fx] = lr

# -----------------------------
# Compute USD-adjusted log returns for all assets
# -----------------------------
asset_returns = {}

for symbol in symbols:
    idx_lr = calculate_log_returns(data[symbol]["close"])

    if symbol in fx_map:
        fx_symbol = fx_map[symbol]
        fx_lr = fx_returns[fx_symbol]

        # Align index and FX by date
        combined = pd.concat([idx_lr, fx_lr], axis=1, join="inner")
        combined.columns = ["idx", "fx"]

        # USD-adjusted return: r_usd = r_index_local + r_fx
        asset_returns[symbol] = combined["idx"] + combined["fx"]
    else:
        # Already in USD terms
        asset_returns[symbol] = idx_lr

# -----------------------------
# Build aligned returns matrix
# -----------------------------
returns = pd.DataFrame(asset_returns).dropna(how="any")

# -----------------------------
# Covariance matrix (daily, log returns)
# -----------------------------
cov_matrix = returns.cov()

# Expected daily returns vector (assumed to be in USD terms already)
expected_returns = np.array([expected_returns_dict[symbol] for symbol in symbols])
expected_returns = expected_returns * returns.std().values  # scale by asset volatilities

# -----------------------------
# Max Sharpe optimization
# -----------------------------
def max_sharpe_ratio(expected_returns, cov_matrix, risk_free_rate=0.0):
    num_assets = len(expected_returns)

    def neg_sharpe(weights):
        port_ret = np.dot(weights, expected_returns)
        port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        return -(port_ret - risk_free_rate) / port_vol

    # Dollar-neutral: sum(weights) = 0
    constraints = [{"type": "eq", "fun": lambda w: np.sum(w)}]

    # Allow long/short
    bounds = [(-1, 1)] * num_assets

    # Start with zero‑sum initial guess
    init_guess = np.ones(num_assets)
    init_guess[: num_assets // 2] = 1
    init_guess[num_assets // 2 :] = -1
    init_guess = init_guess / np.sum(np.abs(init_guess))

    result = minimize(neg_sharpe, init_guess, bounds=bounds, constraints=constraints)

    if not result.success:
        raise ValueError(f"Optimization failed: {result.message}")

    weights = result.x
    daily_ret = np.dot(weights, expected_returns)
    daily_vol = np.sqrt(weights.T @ cov_matrix @ weights)

    return weights, daily_ret, daily_vol

# -----------------------------
# Run optimization
# -----------------------------
weights, daily_port_return, daily_port_vol = max_sharpe_ratio(
    expected_returns, cov_matrix, risk_free_rate=risk_free_rate
)

# -----------------------------
# Export optimal weights
# -----------------------------
weights_df = pd.DataFrame({
    "Symbol": symbols,
    "Weight": weights
})
weights_df.to_csv("aqua_funded_optimal_portfolio_weights.csv", header=False, index=False)
print("\nWeights exported to aqua_funded_optimal_portfolio_weights.csv")

# -----------------------------
# Annualized statistics (using daily log returns)
# -----------------------------
annual_port_return = (1 + daily_port_return)**252 - 1
annual_port_vol = daily_port_vol * np.sqrt(252)
annual_sharpe = (annual_port_return - risk_free_rate) / annual_port_vol

# -----------------------------
# Output
# -----------------------------
print("\nOptimal Portfolio Weights:")
for symbol, weight in zip(symbols, weights):
    print(f"{symbol}: {weight:.4f}")

print("\n--- Portfolio Statistics ---")
print(f"Daily Portfolio Return: {daily_port_return:.4%}")
print(f"Annualized Portfolio Return: {annual_port_return:.4%}")
print(f"Daily Portfolio Volatility: {daily_port_vol:.4%}")
print(f"Annualized Portfolio Volatility: {annual_port_vol:.4%}")
print(f"Annualized Sharpe Ratio: {annual_sharpe:.2f}")

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


Weights exported to aqua_funded_optimal_portfolio_weights.csv

Optimal Portfolio Weights:
AUDUSD: -0.7600
EURUSD: 0.5506
GBPUSD: 0.3116
NZDUSD: -0.1936
USDJPY: 0.2021
USDCHF: 0.4435
USDCAD: -0.7390
XAUUSD: 0.1546
BRENT: 0.0329
SPX500: -0.9987
US30: 0.1064
NAS100: 0.5407
GER40: 0.0491
JPN225: 0.0695
UK100: 0.2288
AUS200: 0.0015

--- Portfolio Statistics ---
Daily Portfolio Return: 1.1767%
Annualized Portfolio Return: 1806.8499%
Daily Portfolio Volatility: 0.2418%
Annualized Portfolio Volatility: 3.8386%
Annualized Sharpe Ratio: 470.71


True

### Volatility Targeted Weights using EWMA estimates

In [22]:
import pandas as pd
import numpy as np

# --- Load weights from comma-separated file ---
weights_df = pd.read_csv(
    "aqua_funded_optimal_portfolio_weights.csv",
    sep=",",
    header=None,
    engine="python",
    encoding="utf-8-sig"
)

weights_df.columns = ["asset", "weight"]

# Align weights to returns.columns order
weights_df = weights_df.set_index("asset").reindex(returns.columns)

# Extract weight vector
weights = weights_df["weight"].values

print("Loaded weights (aligned):")
print(weights)
print("\nAsset order:")
print(list(returns.columns))

# ============================================================
# --- EWMA COVARIANCE MATRIX ---
# ============================================================

lambda_ = 0.94  # RiskMetrics default
rets = returns.values
n = rets.shape[1]

# Start with sample covariance as initial value
ewma_cov = returns.cov().values.copy()

# Iterate through returns to update EWMA covariance
for t in range(1, len(rets)):
    r = rets[t-1].reshape(-1, 1)
    ewma_cov = lambda_ * ewma_cov + (1 - lambda_) * (r @ r.T)

# Portfolio EWMA volatility
daily_returns = returns @ weights
ewma_daily_vol = np.sqrt(weights.T @ ewma_cov @ weights)
ewma_annual_vol = ewma_daily_vol * np.sqrt(252)
ewma_annual_return = daily_returns.mean() * 252  

print("\n--- EWMA VOLATILITY FORECAST ---")
print(f"EWMA Daily Volatility: {ewma_daily_vol:0.2%}")
print(f"EWMA Annual Volatility: {ewma_annual_vol:0.2%}")
print(f"EWMA Daily Portfolio Return: {daily_returns.mean():0.2%}")
print(f"EWMA Annual Portfolio Return: {ewma_annual_return:0.2%}")

# ============================================================
# --- SCALE PORTFOLIO TO TARGET VOLATILITY USING EWMA ---
# ============================================================

target_vol = 0.10  # 10% annual volatility

scale_factor = target_vol / ewma_annual_vol
scaled_weights = weights * scale_factor

# Recompute volatility after scaling
scaled_daily_vol = np.sqrt(scaled_weights.T @ ewma_cov @ scaled_weights)
scaled_annual_vol = scaled_daily_vol * np.sqrt(252)

# Compute scaled returns
scaled_daily_returns = returns @ scaled_weights
scaled_avg_daily_return = scaled_daily_returns.mean()
scaled_annual_return = scaled_avg_daily_return * 252

print(f"\n--- SCALED PORTFOLIO (Target Vol = {target_vol:.0%}) ---")
print(f"Scale Factor: {scale_factor:.4f}")
print(f"Scaled Daily Volatility (EWMA): {scaled_daily_vol:0.2%}")
print(f"Scaled Annual Volatility (EWMA): {scaled_annual_vol:0.2%}")
print(f"Scaled Daily Portfolio Return: {scaled_avg_daily_return:0.2%}")
print(f"Scaled Annual Portfolio Return: {scaled_annual_return:0.2%}")

print("\nScaled Weights:")
for asset, w in zip(returns.columns, scaled_weights):
    print(f"{asset}: {w:.6f}")

# --- EXPORT SCALED WEIGHTS TO CSV ---
scaled_df = pd.DataFrame({
    "asset": returns.columns,
    "scaled_weight": scaled_weights
})

scaled_df.to_csv("aqua_funded_scaled_weights.csv", index=False)
print("\nScaled weights exported to aqua_funded_scaled_weights.csv")

Loaded weights (aligned):
[-0.76000083  0.55058928  0.31160934 -0.19358279  0.20208646  0.44346385
 -0.73897922  0.15459738  0.03291797 -0.9987272   0.10638614  0.5407257
  0.04913331  0.06947293  0.22883973  0.00146795]

Asset order:
['AUDUSD', 'EURUSD', 'GBPUSD', 'NZDUSD', 'USDJPY', 'USDCHF', 'USDCAD', 'XAUUSD', 'BRENT', 'SPX500', 'US30', 'NAS100', 'GER40', 'JPN225', 'UK100', 'AUS200']

--- EWMA VOLATILITY FORECAST ---
EWMA Daily Volatility: 0.27%
EWMA Annual Volatility: 4.37%
EWMA Daily Portfolio Return: 0.07%
EWMA Annual Portfolio Return: 17.19%

--- SCALED PORTFOLIO (Target Vol = 10%) ---
Scale Factor: 2.2908
Scaled Daily Volatility (EWMA): 0.63%
Scaled Annual Volatility (EWMA): 10.00%
Scaled Daily Portfolio Return: 0.16%
Scaled Annual Portfolio Return: 39.37%

Scaled Weights:
AUDUSD: -1.740983
EURUSD: 1.261271
GBPUSD: 0.713824
NZDUSD: -0.443453
USDJPY: 0.462933
USDCHF: 1.015871
USDCAD: -1.692828
XAUUSD: 0.354146
BRENT: 0.075407
SPX500: -2.287849
US30: 0.243706
NAS100: 1.238675
GE

### Lot Sizing

In [23]:
import math
import pandas as pd
import MetaTrader5 as mt5
import numpy as np

# ============================
# USER CONFIG
# ============================
EQUITY = 200000.90  # set your account equity

# FX-exempt symbols (no price in formula)
FX_EXEMPT = ["USDJPY", "USDCHF", "USDCAD"]

# Global index → FX mapping
INDEX_FX_MAP = {
    "GER40": "EURUSD",
    "JPN225": "USDJPY",
    "UK100": "GBPUSD",
    "AUS200": "AUDUSD"
}

# ============================
# MT5 INITIALIZATION
# ============================
mt5.initialize()

def get_latest_price(symbol):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return None
    return tick.ask

def fetch_prices(assets):
    return {a: get_latest_price(a) for a in assets}

def fetch_index_fx_rates():
    return {idx: get_latest_price(fx) for idx, fx in INDEX_FX_MAP.items()}

# ============================
# LOAD CSV FILES
# ============================
contract_df = pd.read_csv("aqua_funded_contract_size.csv")
weights_df = pd.read_csv("aqua_funded_scaled_weights.csv")

# Normalize column names
contract_df.columns = contract_df.columns.str.strip().str.lower()
weights_df.columns = weights_df.columns.str.strip().str.lower()

# Rename scaled_weight → weight
weights_df.rename(columns={"scaled_weight": "weight"}, inplace=True)

# Merge contract sizes + weights
df = contract_df.merge(weights_df, on="asset", how="left")

# ============================
# FETCH PRICES FROM MT5
# ============================
all_assets = df["asset"].tolist()
latest_prices = fetch_prices(all_assets)
index_fx_rates = fetch_index_fx_rates()

df["latest_price"] = df["asset"].map(latest_prices)

# ============================
# LOT SIZE CALCULATION
# ============================
def compute_lot(row):
    asset = row["asset"]
    weight = row["weight"]
    contract_size = row["contract_size"]
    price = row["latest_price"]

    # Missing weight or contract size
    if pd.isna(weight) or pd.isna(contract_size):
        return None

    # Contract size cannot be zero
    if contract_size == 0:
        return None

    # Rule 1: FX-exempt assets
    if asset in FX_EXEMPT:
        return (weight * EQUITY) / contract_size

    # Missing or zero price
    if price is None or price == 0:
        return None

    # Rule 2: Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        # Invert FX rate for JP225
        if asset in ["JPN225"]:
            fx_rate = 1 / fx_rate

        price = price * fx_rate

        # After conversion, price still cannot be zero
        if price == 0:
            return None

    # Final safety check
    denominator = price * contract_size
    if denominator == 0:
        return None

    return (weight * EQUITY) / denominator

from decimal import Decimal, ROUND_HALF_UP

df["lot_size"] = df.apply(compute_lot, axis=1)

df["lot_size"] = df["lot_size"].apply(
    lambda x: Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)
)




# ============================
# OUTPUT
# ============================
print(df[["asset", "weight", "contract_size", "latest_price", "lot_size"]].dropna())
df[["asset", "weight", "contract_size", "latest_price", "lot_size"]]\
    .dropna()\
    .to_csv("aqua_funded_lot_sizes_output.csv", index=False)

# ============================
# GROSS TOTAL LOT SIZE
# ============================
df["abs_lot_size"] = df["lot_size"].abs()
gross_total_lot_size = df["abs_lot_size"].sum()

print(f"\nGross Total Lot Size (absolute): {gross_total_lot_size:.4f}")
mt5.shutdown()

     asset    weight  contract_size  latest_price lot_size
0   AUDUSD -1.740983         100000       0.67031    -5.19
1   EURUSD  1.261271         100000       1.16660     2.16
2   GBPUSD  0.713824         100000       1.34356     1.06
3   NZDUSD -0.443453         100000       0.57477    -1.54
4   USDJPY  0.462933         100000     157.59700     0.93
5   USDCHF  1.015871         100000       0.79768     2.03
6   USDCAD -1.692828         100000       1.38870    -3.39
8   XAUUSD  0.354146            100    4578.05000     0.15
10   BRENT  0.075407            100      63.42000     2.38
11  SPX500 -2.287849             10    6935.90000    -6.60
12    US30  0.243706              1   49311.00000     0.99
13  NAS100  1.238675             10   25600.14000     0.97
14   GER40  0.112553              1   25229.80000     0.76
15  JPN225  0.159146            100   53831.00000     0.93
16   UK100  0.524218              1   10115.90000     7.71
17  AUS200  0.003363              1    8749.00000     0.

True

In [26]:
import MetaTrader5 as mt5
import pandas as pd

# exact sequence you want
symbols = [
    "AUDUSD","EURUSD","GBPUSD","NZDUSD","USDJPY","USDCHF","USDCAD",
    "XAGUSD","XAUUSD","WTI","BRENT",
    "SPX500","US30","NAS100","GER40","JPN225","UK100","AUS200"
]

mt5.initialize()

positions = mt5.positions_get()

if positions is None or len(positions) == 0:
    print("No open positions")
    # create empty table with zeros
    summary = pd.DataFrame({
        "symbol": symbols,
        "buy_size": [0]*len(symbols),
        "sell_size": [0]*len(symbols),
        "net_size": [0]*len(symbols)
    })
else:
    df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())

    df["direction"] = df["type"].map({0: "BUY", 1: "SELL"})

    # aggregate buy/sell per symbol
    agg = df.groupby("symbol").apply(
        lambda x: pd.Series({
            "buy_size": x.loc[x["direction"] == "BUY", "volume"].sum(),
            "sell_size": x.loc[x["direction"] == "SELL", "volume"].sum(),
        })
    )

    agg["net_size"] = agg["buy_size"] - agg["sell_size"]

    # reindex to your exact sequence and fill missing with zeros
    summary = agg.reindex(symbols).fillna(0)

print(summary)

summary.to_csv("open_positions_summary.csv")

        buy_size  sell_size  net_size
symbol                               
AUDUSD      0.00       5.19     -5.19
EURUSD      2.16       0.00      2.16
GBPUSD      1.06       0.00      1.06
NZDUSD      0.00       1.54     -1.54
USDJPY      0.93       0.00      0.93
USDCHF      2.03       0.00      2.03
USDCAD      0.00       3.39     -3.39
XAGUSD      0.00       0.00      0.00
XAUUSD      0.15       0.00      0.15
WTI         0.00       0.00      0.00
BRENT       2.38       0.00      2.38
SPX500      0.00       6.60     -6.60
US30        0.99       0.00      0.99
NAS100      0.97       0.00      0.97
GER40       0.00       0.76     -0.76
JPN225      0.93       0.00      0.93
UK100       7.71       0.00      7.71
AUS200      0.11       0.00      0.11


C:\Users\jessi\AppData\Local\Temp\ipykernel_29020\3719293460.py:30: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg = df.groupby("symbol").apply(


In [29]:
import MetaTrader5 as mt5
from pprint import pprint

mt5.initialize()

symbol = "EURUSD"
info = mt5.symbol_info(symbol)

if info is None:
    print("Symbol not found")
else:
    pprint(info._asdict(), sort_dicts=False)


{'custom': False,
 'chart_mode': 0,
 'select': True,
 'visible': True,
 'session_deals': 0,
 'session_buy_orders': 0,
 'session_sell_orders': 0,
 'volume': 0,
 'volumehigh': 0,
 'volumelow': 0,
 'time': 1768190480,
 'digits': 5,
 'spread': 3,
 'spread_float': True,
 'ticks_bookdepth': 0,
 'trade_calc_mode': 0,
 'trade_mode': 4,
 'start_time': 0,
 'expiration_time': 0,
 'trade_stops_level': 10,
 'trade_freeze_level': 0,
 'trade_exemode': 2,
 'swap_mode': 1,
 'swap_rollover3days': 3,
 'margin_hedged_use_leg': False,
 'expiration_mode': 15,
 'filling_mode': 2,
 'order_mode': 63,
 'order_gtc_mode': 0,
 'option_mode': 0,
 'option_right': 0,
 'bid': 1.16577,
 'bidhigh': 1.16712,
 'bidlow': 1.16214,
 'ask': 1.1658,
 'askhigh': 1.16716,
 'asklow': 1.16221,
 'last': 0.0,
 'lasthigh': 0.0,
 'lastlow': 0.0,
 'volume_real': 0.0,
 'volumehigh_real': 0.0,
 'volumelow_real': 0.0,
 'option_strike': 0.0,
 'point': 1e-05,
 'trade_tick_value': 1.0,
 'trade_tick_value_profit': 1.0,
 'trade_tick_value_loss